# Module 5 - Programming Assignment

## Directions

1. Change the name of this file to be your JHED id as in `jsmith299.ipynb`. Because sure you use your JHED ID (it's made out of your name and not your student id which is just letters and numbers).
2. Make sure the notebook you submit is cleanly and fully executed. I do not grade unexecuted notebooks.
3. Submit your notebook back in Blackboard where you downloaded this file.

*Provide the output **exactly** as requested*

## Solving Normal Form Games

In [1]:
from typing import List, Tuple, Dict, Callable

In the lecture we talked about the Prisoner's Dilemma game, shown here in Normal Form:

Player 1 / Player 2  | Defect | Cooperate
------------- | ------------- | -------------
Defect  | -5, -5 | -1, -10
Cooperate  | -10, -1 | -2, -2

where the payoff to Player 1 is the left number and the payoff to Player 2 is the right number. We can represent each payoff cell as a Tuple: `(-5, -5)`, for example. We can represent each row as a List of Tuples: `[(-5, -5), (-1, -10)]` would be the first row and the entire table as a List of Lists:

In [2]:
prisoners_dilemma = [
 [( -5, -5), (-1,-10)],
 [(-10, -1), (-2, -2)]]

prisoners_dilemma

[[(-5, -5), (-1, -10)], [(-10, -1), (-2, -2)]]

in which case the strategies are represented by indices into the List of Lists. For example, `(Defect, Cooperate)` for the above game becomes `prisoners_dilemma[ 0][ 1]` and returns the payoff `(-1, -10)` because 0 is the first row of the table ("Defect" for Player 1) and 1 is the 2nd column of the row ("Cooperate" for Player 2).

For this assignment, you are going write a function that uses Successive Elimination of Dominated Strategies (SEDS) to find the **pure strategy** Nash Equilibrium of a Normal Form Game. The function is called `solve_game`:

```python
def solve_game( game: List[List[Tuple]], weak=False) -> Tuple:
    pass # returns strategy indices of Nash equilibrium or None.
```

and it takes two parameters: the game, in a format that we described earlier and an optional boolean flag that controls whether the algorithm considers only **strongly dominated strategies** (the default will be false) or whether it should consider **weakly dominated strategies** as well.

It should work with game matrices of any size and it will return the **strategy indices** of the Nash Equilibrium. If there is no **pure strategy** equilibrium that can be found using SEDS, return `None`.


<div style="background: mistyrose; color: firebrick; border: 2px solid darkred; padding: 5px; margin: 10px;">
Do not return the payoff. That's not useful. Return the strategy indices, any other output is incorrect.
</div>

As before, you must provide your implementation in the space below, one Markdown cell for documentation and one Code cell for implementation, one function and assertations per Codecell.


---

<a id="is_losing_move"></a>
## is_losing_move

Checks the the provided move against all provided moves to see if the move is a losing move or not.

Variables
* **move** List[int]: one move which is a list of 'cost' (integer) values
* **moves** List[List[int]]: list of moves

**returns** bool: True if the 'move' is a losing move

In [3]:
def is_losing_move(move: List[int], moves: List[List[int]], weak: bool=False) -> bool:
    for m in moves:
        gt_count = 0
        for i in range(len(move)):
            if move == m:
                continue
            if move[i] < m[i]:
                gt_count += 1
        winning_to_beat = len(move) - 1
        if weak == True:
            winning_to_beat = (len(move) / 2)
        if gt_count > winning_to_beat:
            return True
    return False

In [4]:
assert is_losing_move([-5, -1], [[-5, -1], [-10, -2]]) == False
assert is_losing_move([-10, -2], [[-5, -1], [-10, -2]]) == True
assert is_losing_move([-10, -2], [[-10, -1], [-10, -2]], True) == False
complex =  [
 [(1,3), (3,5), (6,8)],
 [(2,2), (4,4), (7,6)],
 [(3,1), (5,3), (8,7)]]
assert is_losing_move([2,4,7], [[1,3,6],[2,4,7],[3,5,8]]) == True

<a id="get_player_moves"></a>
## get_player_moves

This goes through the game board and generate the lists of moves from the players based off the tuples provided.

Variables
* **game** List[List[Tuple]]: game board full of cost tuples

**returns** two lists of player moves generated from the cost row/column tuples

In [5]:
def get_player_moves(game: List[List[Tuple]]):
    p1_moves = []
    p2_moves = []
    for row in range(len(game)):
        move = []
        for t in game[row]:
            move.append(t[0])
        p1_moves.append(move)
    for col in range(len(game[0])):
        move = []
        for t in range(len(game)):
            move.append(game[t][col][1])
        p2_moves.append(move)
    return p1_moves, p2_moves

In [6]:
assert get_player_moves(prisoners_dilemma) == ([[-5, -1], [-10, -2]], [[-5, -1], [-10, -2]])
assert get_player_moves([[(0, 1), (0,1)],[(2, 3), (4, 5)]]) == ([[0, 0], [2, 4]], [[1, 3], [1, 5]])
complex =  [
 [(1,3), (3,5), (6,8)],
 [(2,2), (4,4), (7,6)],
 [(3,1), (5,3), (8,7)]]
assert get_player_moves(complex) == ([[1,3,6],[2,4,7],[3,5,8]], [[3,2,1],[5,4,3],[8,6,7]])

<a id="solve_game"></a>
## solve_game

(not really sure how to get this down to <= 20 lines without just arbitrarily breaking the code up for what feels to me like no good reason so i'm leaving it at 28 lines)

This does the following:
- gets all player moves
- checks for which moves are losing moves
- builds a solution grid based off what moves are dominated
- finds the nash equilibrium based off if if exists

Variables
* **game** List[List[Tuple]]: game board full of cost tuples
* **weak** bool: this indicates if we should consider 'weak' strategies

**returns** None or the index of the nash equilibrium strategy

In [7]:
def solve_game(game: List[List[Tuple]], weak: bool=False) -> Tuple:
    p1_moves, p2_moves = get_player_moves(game)
    losing_p1_moves, losing_p2_moves = [], []
    for i in range(len(p1_moves)):
        if is_losing_move(p1_moves[i], p1_moves, weak):
            losing_p1_moves.append(i)
    for i in range(len(p2_moves)):
        if is_losing_move(p2_moves[i], p2_moves, weak):
            losing_p2_moves.append(i)
    solution = []
    for r in range(len(game)):
        row = []
        for c in range(len(game[r])):
            t0, t1 = '', ''
            if r in losing_p1_moves:
                t0 = 'L'
            if c in losing_p2_moves:
                t1 = 'L'
            row.append((t0, t1))
        solution.append(row)
    possible_wins = []
    for r in range(len(solution)):
        for c in range(len(solution[r])):
            if solution[r][c][0] == '' and solution[r][c][1] == '':
                possible_wins.append((r,c))
    if len(possible_wins) == 1 or (weak == True and len(possible_wins) > 0):
        return possible_wins[0]
    return None

## Additional Directions

Create three games as described and according to the following:

1. Your games must be created and solved "by hand".
2. The strategy pairs must **not** be on the main diagonal (0, 0), (1, 1), or (2, 2). And the solution cannot be the same for both Game 1 and Game 2.
3. Make sure you fill out the Markdown ("?") with your game as well as the solution ("?").
4. Remember, **do not return the payoff**, return the strategy indices.

For games that can be solved with *weak* SEDS, there may be more than one solution. You only need to return the first solution found. However, if you would like to return all solutions, you can implement `solve_game` as state space search.

### Test Game 1. Create a 3x3 two player game

**that can only be solved using the Successive Elimintation of Strongly Dominated Strategies**

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | 0,1 | 1,10 | 3,5 |
|1  | 1,3 | 3,5 | 5,0 |
|2  | 8,0 | **7,9** | 6,7 |

**Solution:**? (strategy indices)

In [8]:
test_game_1 = [[(0,1), (1,10), (3,5)], [(1,3), (3,5), (5,0)], [(8,0), (7,9), (6,7)]]

solution = solve_game(test_game_1)
print(solution)

(2, 1)


In [9]:
assert solution == (2,1)

### Test Game 2. Create a 3x3 two player game

**that can only be solved using the Successive Elimintation of Weakly Dominated Strategies**

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | 0,1 | 1,3 | 3,5 |
|1  | 7,3 | **3,5** | 5,0 |
|2  | 3,0 | 2,9 | 1,7 |

**Solution:**? (strategy indices)

In [10]:
test_game_2 = [[(0,1), (1,3), (3,5)], [(1,3), (3,5), (5,0)], [(3,0), (2,9), (1,7)]]

strong_solution = solve_game(test_game_2)
weak_solution = solve_game(test_game_2, weak=True)

In [11]:
assert strong_solution == None
assert weak_solution == (1,1)

### Test Game 3. Create a 3x3 two player game

**that cannot be solved using the Successive Elimintation of Dominated Strategies at all**

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | 7,1 | 4,3 | 3,5 |
|1  | 7,3 | 3,5 | 5,0 |
|2  | 3,5 | 2,2 | 1,4 |

**Solution:** None

In [12]:
test_game_3 = [[(7,1), (4,3), (3,5)], [(7,3), (3,5), (5,0)], [(3,5), (2,2), (1,4)]]

strong_solution = solve_game( test_game_3)
weak_solution = solve_game( test_game_3, weak=True)

In [13]:
assert strong_solution == None
assert weak_solution == None

## Before You Submit...

1. Did you provide output exactly as requested? **Don't forget to fill out the Markdown tables with your games**.
2. Did you re-execute the entire notebook? ("Restart Kernel and Rull All Cells...")
3. If you did not complete the assignment or had difficulty please explain what gave you the most difficulty in the Markdown cell below.
4. Did you change the name of the file to `jhed_id.ipynb`?

Do not submit any other files.